# Ejecutar pipelines

In [ ]:
pip show azure-ai-ml

## Conectar a workspace

In [ ]:
# conectar
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

print(f"Conectado al Workspace: {ml_client.workspace_name}")

## Cargar los componentes

In [ ]:
from azure.ai.ml import load_component

ing_prep_data = ml_client.components.get(name="ing_prep_data_housing_ric", version=2)
train_model_housing = ml_client.components.get(name="train_model_housing_ric", version=2)
eval_model_housing = ml_client.components.get(name="eval_model_housing_ric", version=10)

## Build del pipeline

In [ ]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline


@pipeline(name="house_pricing_pipeline", description="Pipeline to train a house pricing model")
def house_pricing(input_data, alpha):
    clean_data = ing_prep_data(input_data=input_data)
    train_model = train_model_housing(input_data=clean_data.outputs.output_data, alpha=alpha)
    eval_model = eval_model_housing(input_data=train_model.outputs.output_data, model_input=train_model.outputs.model_output)

    return {
        "pipeline_job_transformed_data": clean_data.outputs.output_data,
        "pipeline_job_trained_model": train_model.outputs.model_output,
        "pipeline_job_output_data": train_model.outputs.output_data,
    }

In [ ]:
# definir job de pipeline
pipeline_job = house_pricing(Input(type=AssetTypes.URI_FILE, path="azureml:housing_prices:2"), alpha=0.5)

# change the output mode
pipeline_job.outputs.pipeline_job_transformed_data.mode = "upload"
pipeline_job.outputs.pipeline_job_trained_model.mode = "upload"

# set pipeline level compute
pipeline_job.settings.default_compute = "ricardoenm2"

# set pipeline level datastore
pipeline_job.settings.default_datastore = "training_data_ric"
pipeline_job.settings.force_rerun = True

In [ ]:
print(pipeline_job)

## Enviar el job de pipeline

In [ ]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job,
    experiment_name="pipeline_housing_prices_ric",
    display_name="housing_prices_pipeline_job_ric"
)

pipeline_job